In [10]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from itertools import combinations_with_replacement
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression, r_regression, mutual_info_regression


In [63]:
env_name = 'Acrobot-v1'


In [64]:
if env_name == 'Pendulum-v1':
    agent_name =  'sb_sac'
    algo_name= 'maxent'
    group_name = 'test_run1'
    data_run = 'run_scarlet-glade-713'

    dimensions = 3
    len_traj = 200

    seed = 408      
    exp_n = 1

    data_exp = f'{env_name}___group_{group_name}/{data_run}'

    data_path = f'../../results/{data_exp}/files/'

    expert_trajs = np.load(data_path + 'expert_trajs_200.npy')
    expert_ts = np.load(data_path + 'expert_ts_200.npy')
    expert_rs = np.load(data_path + 'expert_rs_200.npy')

    
elif env_name == 'CartPole-v1':
    agent_name =  'sb_sac'
    algo_name= 'maxent'
    group_name = 'test_run2'
    data_run = 'run_flowing-mountain-300'

    dimensions = 4
    len_traj = 500
    
    seed = 408      
    exp_n = 1

    data_exp = f'{env_name}___group_{group_name}/{data_run}'

    data_path = f'../../results/{data_exp}/files/'

    expert_trajs = np.load(data_path + 'expert_trajs_200.npy')[:50000]
    expert_ts = np.load(data_path + 'expert_ts_200.npy')[:50000]
    expert_rs = np.load(data_path + 'expert_rs_200.npy')[:50000]

    
elif env_name == 'Acrobot-v1':
    agent_name =  'sb_sac'
    algo_name= 'maxent'
    group_name = 'test_run2'
    data_run = 'run_splendid-blaze-28'

    seed = 408      
    exp_n = 1    
    
    dimensions = 6
    len_traj = 500

    data_exp = f'{env_name}___group_{group_name}/{data_run}'

    data_path = f'../../results/{data_exp}/files/'


    expert_trajs = np.load(data_path + 'expert_trajs_100.npy')
    expert_ts = np.load(data_path + 'expert_ts_100.npy')
    expert_rs = np.load(data_path + 'expert_rs_100.npy')


else:
    NotImplementedError
    
    
print(expert_trajs.shape, expert_rs.shape)


(11216, 6) (11216,)


In [65]:
import numpy as np

import numpy as np

def divide_1d_array_into_chunks(data_array, time_indexes):
    """
    Divide a 1D array into chunks based on sequential time indexes.

    Parameters:
    - data_array: The 1D array of shape (N,).
    - time_indexes: The array representing sequential time indexes for each point.

    Returns:
    - chunks: A list of chunks, where each chunk is a 1D array.
    """
    chunks = []
    start_index = 0

    for i in range(1, len(time_indexes)):
        if time_indexes[i] < time_indexes[i - 1]:
            # If the time index resets, create a new chunk
            chunks.append(data_array[start_index:i])
            start_index = i

    # Add the last chunk
    chunks.append(data_array[start_index:])

    return chunks

def divide_into_chunks(data_array, time_indexes):
    """
    Divide a 2D array into chunks based on sequential time indexes.

    Parameters:
    - data_array: The 2D array of shape (N, d).
    - time_indexes: The array representing sequential time indexes for each point.

    Returns:
    - chunks: A list of chunks, where each chunk is a 2D array.
    """
    chunks = []
    start_index = 0

    for i in range(1, len(time_indexes)):
        if time_indexes[i] < time_indexes[i - 1]:
            # If the time index resets, create a new chunk
            chunks.append(data_array[start_index:i, :])
            start_index = i

    # Add the last chunk
    chunks.append(data_array[start_index:, :])

    return chunks


# Divide into chunks
traj_chunks = divide_into_chunks(expert_trajs, expert_ts)




In [74]:
# Configs
load_data = True
normalize = True
use_uniform = False
run_regression = True
max_iter = 10
n_trajs = 300
num_points = 10000
# ratio = 0.8


In [75]:
if normalize:
    sc = StandardScaler()
    expert_trajs_std = sc.fit_transform(expert_trajs)
    print('mean : ', sc.mean_)
    print('std  : ', sc.scale_)

mean :  [ 5.13107037e-01  5.17043209e-04  1.85796909e-01  1.07996521e-03
 -2.94198878e-02  7.31103075e-01]
std  :  [0.54347388 0.66434708 0.68963981 0.6999109  2.41187855 4.21353822]


In [76]:
# Generate polynomials of first and second degree
var_type=''.join(['c' for _ in range(dimensions)])
variables = [f'f_{i}' for i in range(dimensions)]
first_degree = variables
second_degree = [f"{x} * {y}" for x, y in combinations_with_replacement(variables, 2)]

# Combine both first and second degree polynomials
all_polynomials = first_degree + second_degree

# print('\nFeatures: ', all_polynomials)
num_feats = len(all_polynomials)

In [77]:
all_polynomials

['f_0',
 'f_1',
 'f_2',
 'f_3',
 'f_4',
 'f_5',
 'f_0 * f_0',
 'f_0 * f_1',
 'f_0 * f_2',
 'f_0 * f_3',
 'f_0 * f_4',
 'f_0 * f_5',
 'f_1 * f_1',
 'f_1 * f_2',
 'f_1 * f_3',
 'f_1 * f_4',
 'f_1 * f_5',
 'f_2 * f_2',
 'f_2 * f_3',
 'f_2 * f_4',
 'f_2 * f_5',
 'f_3 * f_3',
 'f_3 * f_4',
 'f_3 * f_5',
 'f_4 * f_4',
 'f_4 * f_5',
 'f_5 * f_5']

In [78]:
len(all_polynomials)

27

In [79]:
from sklearn.neighbors import KernelDensity
rng = np.random.RandomState(42)

# features
feats = np.zeros((num_points, num_feats )) # 

for idx in range(num_points):

    curr_point = expert_trajs_std[idx]
    
    # find features
    second_degree = [x * y for x, y in combinations_with_replacement(curr_point, 2)]

    # Combine both first and second degree polynomials
    feats[idx, :] = np.array(curr_point.tolist() + second_degree)

feats.shape

(10000, 27)

In [80]:
# ALL
selected_feats = [i for i in range(len(all_polynomials))]
data = feats[:, selected_feats]

kde = KernelDensity(kernel='gaussian', bandwidth=0.05).fit(data)
logP_s = kde.score_samples(data[:, :])

In [81]:
logP_s_chunks = divide_1d_array_into_chunks(logP_s, expert_ts[:num_points])
logP_s_chunks[0].shape, logP_s_chunks[1].shape, logP_s_chunks[2].shape


((78,), (96,), (86,))

In [82]:
feat_chunks = divide_into_chunks(feats, expert_ts[:num_points])
feat_chunks[0].shape, feat_chunks[1].shape, feat_chunks[2].shape


((78, 27), (96, 27), (86, 27))

### Construct regression problem with density and features
$$ log(P(s)) \propto \phi(s) $$ 

#### <center> 1. Linear model: $$ log(P(s)) = \theta^T  * \phi(s) $$ </center>

In [83]:
logP_s_traj = []
X = []
for i in range(len(logP_s_chunks)):
    
    logP_s_traj.append(logP_s_chunks[i].sum())
    X.append(feat_chunks[i].sum(axis=0))
    

logP_s_traj = np.array(logP_s_traj)
X = np.array(X)

y = logP_s_traj

In [84]:
X.shape, y.shape

((89, 27), (89,))

In [85]:

from sklearn.feature_selection import f_regression, r_regression, mutual_info_regression, chi2
np.random.seed(0)

n = num_feats

# Calculate metrics
# f_reg = np.abs(r_regression(X, y))
# f_reg /= np.max(f_reg)

f_test, _ = np.abs(f_regression(X, y))
f_test /= np.max(f_test)

mi = mutual_info_regression(X, y)
mi /= np.max(mi)

plt.figure(figsize=(10, int(len(all_polynomials) * 5 )))
n = num_feats
for i in range(n):
    
    print("index={:.1f}, F={:<{}}, F-test={:.2f}, MI={:.2f}".format(i, all_polynomials[i], 15, f_test[i], mi[i]))

plt.show()


index=0.0, F=f_0            , F-test=0.01, MI=0.15
index=1.0, F=f_1            , F-test=0.01, MI=0.05
index=2.0, F=f_2            , F-test=0.05, MI=0.20
index=3.0, F=f_3            , F-test=0.00, MI=0.00
index=4.0, F=f_4            , F-test=0.00, MI=0.09
index=5.0, F=f_5            , F-test=0.05, MI=0.24
index=6.0, F=f_0 * f_0      , F-test=0.01, MI=0.24
index=7.0, F=f_0 * f_1      , F-test=0.00, MI=0.01
index=8.0, F=f_0 * f_2      , F-test=0.00, MI=0.13
index=9.0, F=f_0 * f_3      , F-test=0.00, MI=0.09
index=10.0, F=f_0 * f_4      , F-test=0.00, MI=0.12
index=11.0, F=f_0 * f_5      , F-test=0.04, MI=0.15
index=12.0, F=f_1 * f_1      , F-test=0.09, MI=0.75
index=13.0, F=f_1 * f_2      , F-test=0.00, MI=0.00
index=14.0, F=f_1 * f_3      , F-test=0.01, MI=0.30
index=15.0, F=f_1 * f_4      , F-test=0.00, MI=0.00
index=16.0, F=f_1 * f_5      , F-test=0.01, MI=0.10
index=17.0, F=f_2 * f_2      , F-test=0.85, MI=0.83
index=18.0, F=f_2 * f_3      , F-test=0.00, MI=0.03
index=19.0, F=f_2 * f_

<Figure size 720x9720 with 0 Axes>

In [86]:
# '''
# 1. sklearn SelectKBest + f_regression/r_regression

# '''

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression

print('f_regression')

for i in range(2,8):
    transformer = SelectKBest(f_regression, k=i).fit(X, y)
    #print(f'    BEST {i} features', transformer.get_feature_names_out())
    mask = transformer.get_support().astype(int)
    selected = [b for a, b in zip(mask, all_polynomials) if a]       
    print(f"    {np.sum(mask)} Features: {selected}")


print('mutual_info_regression')
for i in range(2, 8):
    transformer = SelectKBest(mutual_info_regression, k=i).fit(X, y)
    # print(f'    BEST {i} features', transformer.get_feature_names_out())
    mask = transformer.get_support().astype(int)
    selected = [b for a, b in zip(mask, all_polynomials) if a]       
    print(f"    {np.sum(mask)} Features: {selected}")

f_regression
    2 Features: ['f_2 * f_2', 'f_3 * f_3']
    3 Features: ['f_2 * f_2', 'f_3 * f_3', 'f_5 * f_5']
    4 Features: ['f_1 * f_1', 'f_2 * f_2', 'f_3 * f_3', 'f_5 * f_5']
    5 Features: ['f_5', 'f_1 * f_1', 'f_2 * f_2', 'f_3 * f_3', 'f_5 * f_5']
    6 Features: ['f_5', 'f_1 * f_1', 'f_2 * f_2', 'f_2 * f_5', 'f_3 * f_3', 'f_5 * f_5']
    7 Features: ['f_2', 'f_5', 'f_1 * f_1', 'f_2 * f_2', 'f_2 * f_5', 'f_3 * f_3', 'f_5 * f_5']
mutual_info_regression
    2 Features: ['f_2 * f_2', 'f_3 * f_3']
    3 Features: ['f_1 * f_1', 'f_2 * f_2', 'f_3 * f_3']
    4 Features: ['f_1 * f_1', 'f_2 * f_2', 'f_3 * f_3', 'f_4 * f_4']
    5 Features: ['f_1 * f_1', 'f_2 * f_2', 'f_3 * f_3', 'f_4 * f_4', 'f_4 * f_5']
    6 Features: ['f_1 * f_1', 'f_2 * f_2', 'f_3 * f_3', 'f_4 * f_4', 'f_4 * f_5', 'f_5 * f_5']
    7 Features: ['f_1 * f_1', 'f_1 * f_3', 'f_2 * f_2', 'f_3 * f_3', 'f_4 * f_4', 'f_4 * f_5', 'f_5 * f_5']
